In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import json
import seaborn as sns


In [2]:
data = pd.read_csv(os.path.join('..','data','experiment_data.csv'))

In [ ]:
best = 'Very intuitive, an explanation I could give or expect'
second_best = 'Intuitive'
middle = 'Neither intuitive or unintuitive'
second_worst = 'Unintuitive'
worst = 'Not intuitive at all, not an explanation I would give or expect'

data = data.replace(best,5).replace(second_best,4).replace(middle,3).replace(second_worst,2).replace(worst,1)

In [ ]:
filter_questions = data.iloc[:,1:4]
filter_questions.columns = ['Native_Language', 'Nationality','Gender']
#filter_questions = filter_questions.iloc[4:]

In [ ]:
#filter_questions[filter_questions['Native_Language'] == 'English']
filter_questions['Native_Language'] = filter_questions['Native_Language'].apply(lambda x:x.title())
filter_questions['Nationality'] = filter_questions['Nationality'].apply(lambda x:x.title())

In [ ]:
filter_questions.groupby(by=['Nationality','Native_Language']).agg(['count'])

In [ ]:
cols = [col for col in data.columns if 
        (col != 'Timestamp') and ('Optional' not in col) 
                and (not col.startswith('Do') and (not col.startswith('What'))
                and (not col.startswith('Any')) and (not col.startswith('Unnamed')))]

#drop great grandparent
cols = cols[4:]

In [ ]:
clean_data = data.loc[:,cols]

In [ ]:
all_cols = [
    'aunt_aunt_brother',
    'aunt_grandparent_child',
    'aunt_sister_parent',
    'aunt_aunt_sister',
    'uncle_brother_uncle',
    'uncle_grandparent_child',
    'uncle_parent_brother',
    'uncle_uncle_sister',
    'spouse_spouse',
    'spouse_child_parent',
    'brother_uncle_child',
    'brother_parent_child',
    'brother_sister_brother',
    'brother_aunt_aunt',
    'brother_grandparent_grandparent',
    'brother_parent_parent',
    'brother_sister_sister',
    'brother_uncle_uncle',
    'sister_brother',
    'sister_brother_sister',
    'sister_child_aunt',
    'sister_parent_child',
    'sister_aunt_aunt',
    'sister_brother_brother',
    'sister_grandparent_grandparent',
    'sister_parent_parent',
    'sister_uncle_uncle',
    'grandparent_brother_grandparent',
    'grandparent_grandparent_spouse',
    'grandparent_parent_parent',
    'grandparent_sister_grandparent',
    'grandparent_aunt_child',
    'grandparent_parent_child',
    'grandparent_uncle_child',
    'grandparent_child_child',
    'child_child_sister',
    'child_brother_uncle',
    'child_child_brother',
    'child_parent_grandparent',
    'child_sister_aunt',
    'child_spouse_parent',
    'child_parent',
    'child_aunt_brother',
    'child_brother_parent',
    'child_grandparent_child',
    'child_sister_parent',
    'parent_brother_parent',
    'parent_child_grandparent',
    'parent_parent_spouse',
    'parent_sister_parent',
    'parent_aunt_sister',
    'parent_brother_child',
    'parent_grandparent_parent',
    'parent_sister_child',
    'parent_uncle_brother',
    'parent_child',
    'parent_spouse_child'
]

In [ ]:
#dict((clean_data/5).mean(axis=0))


In [ ]:
clean_data.columns = all_cols
values = (clean_data).mean(axis=0)


In [ ]:
#full = pd.concat([filter_questions,clean_data],axis=1)


In [ ]:
#full.groupby(by=['Nationality','Native_Language']).agg(['mean'])


In [ ]:
predicate_weights = dict((clean_data/5).mean(axis=0))


In [ ]:
with open(os.path.join('..','data','predicate_weights.json'),'w') as f:
    json.dump(predicate_weights,f)
    

In [ ]:
#clean_data.describe()


In [ ]:
replace_dict = {'Français':'French','Française':'French'}

filter_questions = filter_questions.replace(replace_dict)


In [ ]:
# from scipy.stats import spearmanr

# rho, _ = spearmanr(clean_data[filter_questions['Native_Language']=='French'].mean(axis=0),
#                   clean_data[filter_questions['Native_Language']=='English'].mean(axis=0))

# print(rho)


In [ ]:
from scipy.stats import mannwhitneyu

stat, p = mannwhitneyu(clean_data[filter_questions['Native_Language']=='French'].mean(axis=0),
                       clean_data[filter_questions['Native_Language']=='English'].mean(axis=0))

print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Probably the same distribution')
else:
    print('Probably different distributions')
    

In [ ]:
from scipy.stats import ttest_ind

stat, p = ttest_ind(clean_data[filter_questions['Native_Language']=='French'].mean(axis=0),
                       clean_data[filter_questions['Native_Language']=='English'].mean(axis=0))
print('stat=%.3f, p=%.3f' % (stat, p))
if p > 0.05:
    print('Probably the same distribution')
else:
    print('Probably different distributions')
    

In [ ]:
def bar_plot(values):
    
    std_error = np.std(values, ddof=1) / np.sqrt(len(values))
    
    ticks = np.arange(len(values))
    
    explanations = []

    for i in values.index:

        split_str = i.split('_')

        relation = split_str[0]
        
        explanations.append('_'.join(split_str[1:]))

    fig, ax = plt.subplots(figsize=(10,4))

    ax.bar(x=ticks, 
           height=values,
           yerr=std_error,
          capsize=4)
    
    ax.set_xticks(ticks=ticks)
    ax.set_xticklabels(labels=explanations,rotation = (45), fontsize = 10)
    
    ax.set_title(relation.title())
    plt.savefig(os.path.join('..','plots',f"{relation}.pdf"),bbox_inches='tight')
    
    

In [ ]:
bar_plot(values[0:4]) 


In [ ]:
bar_plot(values[4:8])


In [ ]:
bar_plot(values[8:10])


In [ ]:
bar_plot(values[10:18])


In [ ]:
bar_plot(values[18:27])


In [ ]:
bar_plot(values[27:35])


In [ ]:
bar_plot(values[35:46])


In [ ]:
bar_plot(values[46:])


In [3]:
import utils


In [4]:
DATASET = 'paul_dataset'
RULE = 'spouse'


In [5]:
data = np.load(os.path.join('..','data',DATASET+'.npz'))

triples,traces,weights,entities,relations = utils.get_data(data,RULE)


In [6]:
NUM_ENTITIES = len(entities)
NUM_RELATIONS = len(relations)

ent2idx = dict(zip(entities, range(NUM_ENTITIES)))
rel2idx = dict(zip(relations, range(NUM_RELATIONS)))

idx2ent = dict(zip(range(NUM_ENTITIES),entities))
idx2rel = dict(zip(range(NUM_RELATIONS),relations))


In [7]:
triples2idx = utils.array2idx(triples,ent2idx,rel2idx)
traces2idx = utils.array2idx(traces,ent2idx,rel2idx)


In [ ]:
all_traces = np.unique(np.concatenate([triples2idx,traces2idx.reshape(-1,3)],axis=0),axis=0)


In [8]:
relevance_scores = utils.get_relevance_scores(
    traces2idx,
    weights,
    ent2idx['UNK_ENT'],
    rel2idx['UNK_REL']
)


In [ ]:
# flatten_traces = np.unique(traces.reshape(-1,3),axis=0)
# for i in range(len(flatten_traces)):
    
#     is_duplicate = (flatten_traces[i] == flatten_traces).all(axis=1).sum()
    
#     if is_duplicate >= 2:
#         print(i)

In [ ]:
pred_exp = np.array([[3,1,9], [3,1,1]])


In [ ]:
relevance_scores[str(pred_exp[1])]['1_2'] = .4


In [50]:
unk_ent_id = ent2idx['UNK_ENT']
unk_rel_id = rel2idx['UNK_REL']


In [51]:
true_exp = utils.remove_padding_np(
    traces2idx[3],
    unk_ent_id,
    unk_rel_id)


In [90]:
gnn_data = np.load(
    os.path.join('..','data','preds',DATASET,
        'gnn_explainer_'+DATASET+'_'+RULE+'_preds.npz'),allow_pickle=True)


In [96]:
for pred in gnn_data['preds']:
    print(pred.shape)
    

(6, 3)
(6, 3)
(22, 3)
(6, 3)
(6, 3)
(6, 3)
(22, 3)
(6, 3)


In [97]:
gnn_data['preds']


array([array([['<http://example.org/data#FatherPaul>', 'child',
               '<http://example.org/data#SisterPaul>'],
              ['<http://example.org/data#MotherPaul>', 'child',
               '<http://example.org/data#SisterPaul>'],
              ['<http://example.org/data#SisterPaul>', 'parent',
               '<http://example.org/data#FatherPaul>'],
              ['<http://example.org/data#SisterPaul>', 'parent',
               '<http://example.org/data#MotherPaul>'],
              ['<http://example.org/data#BrotherInLawPaul>', 'spouse',
               '<http://example.org/data#SisterPaul>'],
              ['<http://example.org/data#SisterPaul>', 'spouse',
               '<http://example.org/data#BrotherInLawPaul>']], dtype='<U42'),
       array([['<http://example.org/data#FatherPaul>', 'child',
               '<http://example.org/data#BrotherPaul>'],
              ['<http://example.org/data#MotherPaul>', 'child',
               '<http://example.org/data#BrotherPaul>'],
      

In [ ]:
precision = get_g_precision(pred_exp,true_exp,relevance_scores)
precision


In [ ]:
recall = get_g_recall(pred_exp,true_exp,relevance_scores)
recall


In [ ]:
utils.f1(precision,recall)


In [ ]:
def get_g_precision(pred_exp,true_exp,relevance_scores):
    
    n = len(pred_exp)

    if np.issubdtype(true_exp.dtype, np.integer):
        relation_key = '_'.join([str(i) for i in true_exp[:,1]])
    else:
        relation_key = '_'.join(true_exp[:,1])
        
    g_precision = 0.0
    
    for i in range(n):
        
        triple = pred_exp[i]
        
        rel_score = relevance_scores.get(str(triple),0).get(relation_key,0)
        #rel_score = relevance_scores[str(triple)][relation_key]
        
        g_precision += rel_score
        
    return g_precision / n

def get_g_recall(pred_exp,true_exp,relevance_scores):
    
    pred_relevance = 0.0
    total_relevance = 0.0
    
    for d in relevance_scores.values():
        total_relevance += sum(d.values())
    
    if np.issubdtype(true_exp.dtype, np.integer):
        relation_key = '_'.join([str(i) for i in true_exp[:,1]])
    else:
        relation_key = '_'.join(true_exp[:,1])
    
    for i in range(len(pred_exp)):
        
        triple = pred_exp[i]
        
        rel_score = relevance_scores.get(str(triple),0).get(relation_key,0)
        #rel_score = relevance_scores[str(triple)][relation_key]
        
        pred_relevance += rel_score
        
    return pred_relevance / total_relevance